1) 이 문단에서 parameters(파라미터)란?
파라미터는 모델이 학습하는 동안 조정되는 값
eg) 가중치(weight)와 편향(bias)

2) 손실 함수와 옵티마이저는 모델을 어떻게 학습시킬지를 설명하는 도구
옵티마이저(optimizer)는 모델의 파라미터(가중치와 편향)를 조정하여 손실을 줄이는 역할을 함. 

3) 손실 함수를 통해 모델이 얼마나 틀렸는지를 측정한 후, 
옵티마이저가 이를 기반으로 모델의 가중치와 편향 같은 파라미터를 업데이트한다. 

4) 하이퍼 파라미터는 모델을 학습시키기 전에 사용자가 설정하는 값
파라미터는 모델이 학습하면서 자동으로 조정되는 값인 반면, 
하이퍼 파라미터는 학습 전에 사용자가 설정

In [16]:
# Linear Regression 모델 생성 및 데이터 준비
import torch
from torch import nn
import matplotlib.pyplot as plt

# create known parameters
weight = 0.7
bias = 0.3

# create input data
start = 0
end = 1
step = 0.02
X = torch.arange(start, end, step).unsqueeze(dim=1)  # (N, 1)으로 변환
y = weight * X + bias

# 데이터 분할
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

# Linear Regression 모델 클래스 정의
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
        self.bias = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.weights * x + self.bias

# 모델 생성
model_0 = LinearRegressionModel()

# 예측 전 X_test 차원 확인 및 변환 (필요할 경우)
print(X_test.shape)  # 2차원인지 확인
if len(X_test.shape) == 1:
    X_test = X_test.unsqueeze(dim=1)  # 1차원일 경우 2차원으로 변환

# 추론 모드로 예측 수행
with torch.inference_mode():
    y_preds = model_0(X_test)

# 예측 결과 출력
print(y_preds)


torch.Size([10, 1])
tensor([[0.3982],
        [0.4049],
        [0.4116],
        [0.4184],
        [0.4251],
        [0.4318],
        [0.4386],
        [0.4453],
        [0.4520],
        [0.4588]])


In [17]:
# 3 Train Models 
# in 
list(model_0.parameters()) # 선형회귀 클래스에서 설정한 weight,bias가 결과로 뜬다. 


[Parameter containing:
 tensor([0.3367], requires_grad=True),
 Parameter containing:
 tensor([0.1288], requires_grad=True)]

In [18]:
# Check out our model's parameters (a parameter is a value that the model sets itself)
model_0.state_dict()

OrderedDict([('weights', tensor([0.3367])), ('bias', tensor([0.1288]))])

In [19]:
# Setup a loss function
loss_fn = nn.L1Loss()

# Setup an optimizer (stochastic gradient descent)
optimizer = torch.optim.SGD(params=model_0.parameters(), # we want to optimize the parameters present in our model
                            lr=0.01) # lr = learning rate = possibly the most important hyperparameter you can set
     

For a regression problem (like ours), a loss function of nn.L1Loss() and an optimizer like torch.optim.SGD() will suffice.

For a classification problem like classifying whether a photo is of a dog or a cat, you'll likely want to use a loss function of nn.BCELoss() (binary cross entropy loss).

### Building a training loop (and a testing loop) in PyTorch

Forward pass: 모델에 데이터를 넣고 예측값을 계산.
Calculate loss: 예측값과 실제 정답을 비교해서 손실을 계산.
Optimizer zero grad: 기울기를 0으로 초기화.
Loss backward: 역전파를 통해 파라미터에 대한 기울기를 계산.
Optimizer step: 기울기를 기반으로 파라미터를 업데이트해 손실을 줄이려는 과정.




1Forward pass (this involves data moving through our model's forward() functions) to make predictions on data - also called forward propagation
- Forward pass는 입력 데이터가 모델을 통과해 예측값을 계산하는 과정
- 입력 데이터 → 모델의 forward() 함수 → 예측값 출력

2Calculate the loss (compare forward pass predictions to ground truth labels)
- loss : **예측값(y_hat)**을 **실제 정답(ground truth labels)**과 비교
- Ground truth labels는 실제 정답값

3Optimizer zero grad 
- Optimizer zero grad는 옵티마이저가 이전에 계산된 기울기(gradient)를 0으로 초기화하는 것
- 기울기(gradient)를 초기화해서 이전 기울기가 다음 계산에 영향을 주지 않도록 하는 것.

4Loss backward - move backwards through the network to calculate the gradients of each of the parameters of our model with respect to the loss 
- 역전파는 손실을 기반으로 모델의 각 파라미터(가중치와 편향)에 대한 기울기(gradient)를 계산하는 과정
- 가중치와 편향을 얼마나 조정해야 하는지를 계산

5Optimizer step - use the optimizer to adjust our model's parameters to try and improve the loss 
- Optimizer step은 옵티마이저(optimizer)가 모델의 파라미터(가중치와 편향)를 업데이트

In [20]:
torch.manual_seed(42)

# An epoch is one loop through the data... 
# (this is a hyperparameter because we've set it ourselves)
epochs = 200 

# Track different values
epoch_count = []
loss_values = []
test_loss_values = []

### Training
# 0. Loop through the data
for epoch in range(epochs):
    # set the model to training mode 
    model_0.train() 
    # train mode in PyTorch sets all parameters 
    # that require gradients to require gradients 
    
    # 1. Forward pass 
    y_pred = model_0(X_train)
    
    # 2. Calculate the loss 
    loss = loss_fn(y_pred, y_train)
    
    # 3. Optimizer zero grad 
    optimizer.zero_grad()
    
    # 4. Perform backpropagation on the loss with respect to the parameters of the model
    loss.backward()
    
    # 5. Step the optimizer (perform gradient descent)
    optimizer.step()
    
    ### Testing 
    model_0.eval() 
    #훈련 중과는 달리 평가나 추론할 때는 기울기 계산이 
    # 필요 없으므로 메모리 절약과 속도 향상을 위해 평가 모드를 사용
    with torch.inference_mode(): 
        #기울기 추적을 비활성화하여 
        # 추론 시에 메모리 사용량을 줄이고 속도를 높이는 모드
        test_pred = model_0(X_test)
        
        # Do the forward pass
        test_pred = model_0(X_test)
        
        # Calculate the loss 
        test_loss = loss_fn(test_pred, y_test)
    
    # Print out What is happenin' 이거 무슨 코드인지 해석좀 해줘
    if epoch % 10 == 0:
        # 에포크(epoch)가 10의 배수일 때마다 
        # 훈련 과정의 진행 상황을 출력하고, 손실 값을 기록
        epoch_count.append(epoch)  # 현재 에포크 저장
        loss_values.append(loss)  # 훈련 손실 저장
        test_loss_values.append(test_loss)  # 테스트 손실 저장
        print(f'Epoch:{epoch}|Loss:{loss}|Test loss:{test_loss}')
        
        # print out model state_dict() 
        print(model_0.state_dict())  # 이거는 이제 200번까지 gd한거 나오는거지?
    
    
#이 코드는 훈련 과정과 테스트 과정을 통해 모델의 성능을 평가하고, 
# 훈련 손실과 테스트 손실을 **에포크(epoch)**마다 기록하여 
# **학습 및 평가 곡선(training and test curves)**을 
# 그릴 수 있는 구조로 되어 있다. 
    
    
        

Epoch:0|Loss:0.31288138031959534|Test loss:0.48106518387794495
OrderedDict([('weights', tensor([0.3406])), ('bias', tensor([0.1388]))])
Epoch:10|Loss:0.1976713240146637|Test loss:0.3463551998138428
OrderedDict([('weights', tensor([0.3796])), ('bias', tensor([0.2388]))])
Epoch:20|Loss:0.08908725529909134|Test loss:0.21729660034179688
OrderedDict([('weights', tensor([0.4184])), ('bias', tensor([0.3333]))])
Epoch:30|Loss:0.053148526698350906|Test loss:0.14464017748832703
OrderedDict([('weights', tensor([0.4512])), ('bias', tensor([0.3768]))])
Epoch:40|Loss:0.04543796554207802|Test loss:0.11360953003168106
OrderedDict([('weights', tensor([0.4748])), ('bias', tensor([0.3868]))])
Epoch:50|Loss:0.04167863354086876|Test loss:0.09919948130846024
OrderedDict([('weights', tensor([0.4938])), ('bias', tensor([0.3843]))])
Epoch:60|Loss:0.03818932920694351|Test loss:0.08886633068323135
OrderedDict([('weights', tensor([0.5116])), ('bias', tensor([0.3788]))])
Epoch:70|Loss:0.03476089984178543|Test loss

In [21]:
import numpy as np
np.array(torch.tensor(loss_values).numpy()), test_loss_values

(array([0.31288138, 0.19767132, 0.08908726, 0.05314853, 0.04543797,
        0.04167863, 0.03818933, 0.0347609 , 0.03132383, 0.0278874 ,
        0.02445896, 0.02102021, 0.01758547, 0.01415539, 0.01071659,
        0.00728353, 0.00385178, 0.00893248, 0.00893248, 0.00893248],
       dtype=float32),
 [tensor(0.4811),
  tensor(0.3464),
  tensor(0.2173),
  tensor(0.1446),
  tensor(0.1136),
  tensor(0.0992),
  tensor(0.0889),
  tensor(0.0806),
  tensor(0.0723),
  tensor(0.0647),
  tensor(0.0565),
  tensor(0.0482),
  tensor(0.0406),
  tensor(0.0323),
  tensor(0.0241),
  tensor(0.0165),
  tensor(0.0082),
  tensor(0.0050),
  tensor(0.0050),
  tensor(0.0050)])

In [22]:
# Saving a model in Pytorch 
# torch.save(), torch.load(), torch.nn.Module.load_state_dict()

# Saving our Pytoch model 
from pathlib import Path 

# 1. Create models directory 
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path
MODEL_NAME = "01_pytorch_workflow_model_0.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model_0.state_dict(),
           f=MODEL_SAVE_PATH)

!ls -l models 
# 폴더 안의 파일 목록과 
#파일 정보를 나열하여 모델이 
# 제대로 저장되었는지 확인하는 명령어

Saving model to: models\01_pytorch_workflow_model_0.pth


'ls'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [23]:
# Loading a PyTorch model


In [24]:
# 6. putting it all together  
# import Pytorch and matplolib
import torch 
from torch import nn 
import matplotlib.pyplot as plt 

# check pytorch version 
torch.__version__


'2.0.1+cpu'

In [25]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


In [ ]:
# 6.1 Data 

In [ ]:
# 6.2 Building a PyTorch Linear model 

In [ ]:
# 6.3 Training 
# Loss function
# Optimizer
# Training loop
# Testing loop

In [ ]:
# 6.4 Making and evaluating predictions 

In [ ]:
# 6.5 Saving and Loading a trained model 